## Audit jakości danych

## 0) Importy (biblioteki)

In [ ]:
import pandas as pd
import numpy as np
import os

#pd.set_option("display.max_columns", 120)
#pd.set_option("display.width", 160)

# A) Przykładowe dane

## A1) Tworzymy mini-tabele: customers / products / orders

In [ ]:
# MINI customers
customers_demo = pd.DataFrame({
    "customer_id": ["C001","C002","C003","C004","C005"],
    "customer_zip": ["00-001", "30-002", "NULL", "", "80-100"],
    "customer_city": ["Warszawa", "Kraków", "?", "Gdańsk", "Gdańsk"],
    "created_at": ["2023-01-05", "2023/02/10", "bad_date", "", None],
})

# MINI products
products_demo = pd.DataFrame({
    "product_id": ["P001","P002","P003","P004","P005"],
    "category": ["beauty", "electronics", "NULL", "home", "?"],
    "price": ["19,99", "49,99", "", "9999", "bad"], 
})

# MINI orders
orders_demo = pd.DataFrame({
    "order_id": ["O0001","O0002","O0003","O0004","O0005"],
    "customer_id": ["C001","C002","C003","C003","C005"],
    "product_id": ["P001","P002","P003","P004","P005"],
    "order_status": ["delivered", "shipped", "?", "canceled", ""],
    "order_purchase_timestamp": ["2023-03-01 10:15:00", "2023/03/05 08:00", "bad_date", "", None],
})

orders_demo = pd.concat([orders_demo, orders_demo.iloc[[0]]], ignore_index=True)

print("Dane gotowe.")


## A2) Ile jest danych i jak wyglądają?

In [ ]:
print("orders_demo:", orders_demo.shape)
display(orders_demo.head())

print("customers_demo:", customers_demo.shape)
display(customers_demo.head())

print("products_demo:", products_demo.shape)
display(products_demo.head())


## A3) Typy kolumn i braki

In [ ]:
print("orders_demo.info()")
orders_demo.info()

print("\ncustomers_demo.info()")
customers_demo.info()

print("\n products_demo.info()")
products_demo.info()


## A4) Braki `NaN`: ile procent braków ma każda kolumna?

In [ ]:
print("orders_demo missing %:")
display((orders_demo.isna().mean()*100).sort_values(ascending=False))

print("\ncustomers_demo missing %:")
display((customers_demo.isna().mean()*100).sort_values(ascending=False))

print("\nproducts_demo missing %:")
display((products_demo.isna().mean()*100).sort_values(ascending=False))


## A5) Duplikaty wierszy

In [ ]:
print("orders_demo duplicated rows:", int(orders_demo.duplicated().sum()))
print("customers_demo duplicated rows:", int(customers_demo.duplicated().sum()))
print("products_demo duplicated rows:", int(products_demo.duplicated().sum()))


## A6) Rozkłady kategorii

In [ ]:
print("orders_demo.order_status:")
display(orders_demo["order_status"].astype("string").value_counts(dropna=False))

print("\nproducts_demo.category:")
display(products_demo["category"].astype("string").value_counts(dropna=False))


## A7) „Braki udawane” (placeholdery): `""`, `"NULL"`, `"?"`

In [ ]:
PLACEHOLDERS = ["", "NULL", "null", "?", "N/A", "NA"]

print("orders_demo.order_status placeholder count:",
      int(orders_demo["order_status"].astype("string").isin(PLACEHOLDERS).sum()))

print("customers_demo.customer_zip placeholder count:",
      int(customers_demo["customer_zip"].astype("string").isin(PLACEHOLDERS).sum()))

print("products_demo.price placeholder count:",
      int(products_demo["price"].astype("string").isin(PLACEHOLDERS).sum()))


## A8) Daty jako tekst 

In [ ]:
s = orders_demo["order_purchase_timestamp"]
before = int(s.isna().sum())

converted = pd.to_datetime(s, errors="coerce")
after = int(converted.isna().sum())

print("braki przed:", before)
print("braki po:", after)
print("NOWE braki po konwersji:", after - before)

print("\nWartości, które się nie dały przekonwertować:")
bad = orders_demo.loc[converted.isna() & s.notna(), ["order_purchase_timestamp"]]
display(bad)


## A9) Liczby jako tekst + przecinek

In [ ]:
s = products_demo["price"].astype("string")
s2 = s.str.replace(",", ".", regex=False)
num = pd.to_numeric(s2, errors="coerce")

print("Ile NaN po konwersji?", int(num.isna().sum()))
print("Przykłady, które nie przeszły:")
display(products_demo.loc[num.isna() & products_demo["price"].notna(), ["price"]])

display(pd.DataFrame({"price_raw": s, "price_num": num}).head(10))


## A10) Zamiana placeholderów na `NaN` (standaryzacja braków)

In [ ]:
products_demo_clean = products_demo.replace(PLACEHOLDERS, np.nan)

print("missing % przed:")
display((products_demo.isna().mean()*100).sort_values(ascending=False))

print("\nmissing % po:")
display((products_demo_clean.isna().mean()*100).sort_values(ascending=False))


# B) OLIST (Kaggle)

In [ ]:
#FOLDER = ""  # np. "olist/"

#orders_file    = FOLDER + "olist_orders_dataset.csv"
#customers_file = FOLDER + "olist_customers_dataset.csv"
#products_file  = FOLDER + "olist_products_dataset.csv"

#print("orders exists?   ", os.path.exists(orders_file), "->", orders_file)
#print("customers exists?", os.path.exists(customers_file), "->", customers_file)
#print("products exists? ", os.path.exists(products_file), "->", products_file)


## B1) Wczytanie Olist

In [ ]:
orders = pd.read_csv("olist_orders_dataset.csv")
customers = pd.read_csv("olist_customers_dataset.csv")
products = pd.read_csv("olist_products_dataset.csv")

print("Gotowe")

## B2) `shape` i `head()`

In [ ]:
print("orders:", orders.shape)
display(orders.head(3))

print("customers:", customers.shape)
display(customers.head(3))

print("products:", products.shape)
display(products.head(3))


In [ ]:
customers["customer_id"].nunique(), customers["customer_unique_id"].nunique()

In [ ]:
customers.groupby("customer_unique_id")["customer_id"].nunique().value_counts().head()

## B3) Braki `NaN`

In [ ]:
print("orders missing % (TOP 10):")
display((orders.isna().mean()*100).sort_values(ascending=False).head(10))

print("\ncustomers missing % (TOP 10):")
display((customers.isna().mean()*100).sort_values(ascending=False).head(10))

print("\nproducts missing % (TOP 10):")
display((products.isna().mean()*100).sort_values(ascending=False).head(10))


In [ ]:
%pip install missingno

In [ ]:
msno.heatmap(products, figsize=(10,6))
plt.title("Korelacja braków danych", fontsize=16)
plt.show()

## B4) Duplikaty wierszy 

In [ ]:
print("orders duplicated rows:", int(orders.duplicated().sum()))
print("customers duplicated rows:", int(customers.duplicated().sum()))
print("products duplicated rows:", int(products.duplicated().sum()))


In [ ]:
print("orders duplicate order_id:", orders.duplicated("order_id").sum())
print("customers duplicate customer_id:", customers.duplicated("customer_id").sum())
print("products duplicate product_id:", products.duplicated("product_id").sum())


## B5) `value_counts` na kilku kolumnach 

In [ ]:
if "order_status" in orders.columns:
    print("order_status:")
    display(orders["order_status"].value_counts(dropna=False))

if "customer_state" in customers.columns:
    print("customer_state (top 20):")
    display(customers["customer_state"].value_counts(dropna=False).head(20))

if "product_category_name" in products.columns:
    print("product_category_name (top 20):")
    display(products["product_category_name"].value_counts(dropna=False).head(20))


## B6) Test daty

In [ ]:
if "order_purchase_timestamp" in orders.columns:
    s = orders["order_purchase_timestamp"]
    before = int(s.isna().sum())
    converted = pd.to_datetime(s, errors="coerce")
    after = int(converted.isna().sum())

    print("NOWE braki po:", after - before)

    bad = orders.loc[converted.isna() & s.notna(), ["order_purchase_timestamp"]].head(10)
    if len(bad) > 0:
        print("Przykłady wartości, które nie przeszły konwersji:")
        display(bad)

In [ ]:
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'], errors='coerce')

orders_by_month = orders.groupby(orders['order_purchase_timestamp'].dt.to_period('M')).size()

plt.figure(figsize=(14, 6))
orders_by_month.plot(kind='line', marker='o', color='green')
plt.title("Liczba zamówień w czasie (miesięcznie)")
plt.xlabel("Miesiąc")
plt.ylabel("Liczba zamówień")
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
import seaborn as sns
plt.figure(figsize=(10, 6))
sns.histplot(products[products['product_weight_g'] < 30000]['product_weight_g'], bins=50, kde=True)
plt.title("Rozkład wagi produktów (do 30kg)")
plt.xlabel("Waga (g)")
plt.show()